In [81]:
import numpy as np
import json
from pylatexenc.latex2text import LatexNodes2Text
from pylatexenc.latexencode import unicode_to_latex
from dataclasses import dataclass


In [82]:
with open('example_inspire_record.json', 'r') as file:
    data = json.load(file)

In [161]:
data['hits']['hits'][0]['metadata']['texkeys']

['Batell:2022xau']

In [321]:
rec = InspireRecord(data['hits']['hits'][14]['metadata'])
unicode_to_latex(rec)

'Arg\\"uelles, Foppiani, Hostert, Phys.Rev.D 105 (2022) 9 095006, 2022, arXiv:2109.03831 [hep-ph].'

In [300]:
rec.authors_

['Arg\\"uelles', 'Foppiani', 'Hostert']

In [262]:
for record in data['hits']['hits']:
    if record['metadata']['document_type'][0] == 'proceedings':
        print(record['metadata'].keys())
    

dict_keys(['report_numbers', 'authors', 'publication_info', 'documents', 'citation_count', 'citation_count_without_self_citations', 'citeable', '$schema', 'keywords', 'references', 'number_of_pages', 'legacy_creation_date', 'preprint_date', 'author_count', 'public_notes', 'earliest_date', 'external_system_identifiers', 'facet_author_name', 'core', 'license', 'journal_title_variants', 'arxiv_eprints', 'referenced_authors_bais', 'figures', 'legacy_version', 'inspire_categories', 'urls', 'first_author', 'control_number', 'dois', 'document_type', 'texkeys', 'abstracts', 'primary_arxiv_category', 'titles', 'imprints', 'curated'])


In [226]:
i=40
print(data['hits']['hits'][i]['metadata']['document_type'][0],'\n')

for key in data['hits']['hits'][i]['metadata'].keys():
    print(key, type(data['hits']['hits'][i]['metadata'][key]))
        



conference paper 

authors <class 'list'>
publication_info <class 'list'>
documents <class 'list'>
citation_count <class 'int'>
citation_count_without_self_citations <class 'int'>
citeable <class 'bool'>
$schema <class 'str'>
keywords <class 'list'>
references <class 'list'>
number_of_pages <class 'int'>
referenced_authors_bais <class 'list'>
legacy_version <class 'str'>
inspire_categories <class 'list'>
legacy_creation_date <class 'str'>
author_count <class 'int'>
urls <class 'list'>
first_author <class 'dict'>
control_number <class 'int'>
dois <class 'list'>
earliest_date <class 'str'>
document_type <class 'list'>
texkeys <class 'list'>
abstracts <class 'list'>
titles <class 'list'>
facet_author_name <class 'list'>
core <class 'bool'>
license <class 'list'>
imprints <class 'list'>
curated <class 'bool'>
journal_title_variants <class 'list'>


C. A. Argüelles, I. Esteban, M. Hostert, Kevin J. Kelly, J. Kopp, P. A. N. Machado, I. Martinez-Soler, Y. F. Perez-Gonzalez
Argüelles, C.A. and Esteban, I. and Hostert, M. and Kelly, Kevin J. and Kopp, J. and Machado, P.A.N. and Martinez-Soler, I. and Perez-Gonzalez, Y.F.
